In [3]:
import json

In [6]:
DATANAME = '../../data/testcube'

In [54]:
index = None
with open(f'{DATANAME}.index', 'rb') as f:
    index = f.read()
    index = index.split('\x03'.encode('ascii'))
    index = filter(lambda i: len(i) > 0, index)
    index = [json.loads(i) for i in index]

In [71]:
list(index[0]['status'].keys())

['webhooks',
 'configfile',
 'mcu',
 'gcode_move',
 'print_stats',
 'virtual_sdcard',
 'pause_resume',
 'display_status',
 'gcode_macro CANCEL_PRINT',
 'gcode_macro PAUSE',
 'gcode_macro RESUME',
 'gcode_macro SET_PAUSE_NEXT_LAYER',
 'gcode_macro SET_PAUSE_AT_LAYER',
 'gcode_macro SET_PRINT_STATS_INFO',
 'gcode_macro _TOOLHEAD_PARK_PAUSE_CANCEL',
 'gcode_macro _CLIENT_EXTRUDE',
 'gcode_macro _CLIENT_RETRACT',
 'gcode_macro HOME',
 'heaters',
 'heater_bed',
 'fan',
 'heater_fan fan1',
 'stepper_enable',
 'tmc2209 stepper_x',
 'tmc2209 stepper_y',
 'tmc2209 stepper_z',
 'tmc2209 extruder',
 'menu',
 'motion_report',
 'query_endstops',
 'idle_timeout',
 'system_stats',
 'manual_probe',
 'toolhead',
 'extruder']

In [60]:
data = None
with open(f'{DATANAME}.json', 'rb') as f:
    data = f.read()
    data = data.split('\x03'.encode('ascii'))
    data = filter(lambda i: len(i) > 0, data)
    data = [json.loads(i) for i in data]

In [97]:
data[50]

{'q': 'status',
 'params': {'eventtime': 147.636227809,
  'status': {'heater_bed': {'temperature': 35.8},
   'toolhead': {'estimated_print_time': 2876.262619975}}}}

# Analyze data

In [149]:
status_keys = set()

for row in data:
    if 'q' not in row or row['q'] != 'status':
        continue
    status_keys |= set(row['params']['status'].keys())

In [150]:
status_keys

{'display_status',
 'extruder',
 'fan',
 'gcode_move',
 'heater_bed',
 'heater_fan fan1',
 'idle_timeout',
 'mcu',
 'motion_report',
 'print_stats',
 'stepper_enable',
 'system_stats',
 'tmc2209 extruder',
 'tmc2209 stepper_x',
 'tmc2209 stepper_y',
 'tmc2209 stepper_z',
 'toolhead',
 'virtual_sdcard'}

In [167]:
others_keys = set()
for row in data:
    if 'q' in row and 'params' in row and row['q'] != 'status':
        others_keys |= row['params'].keys()
print(others_keys)

{'start_mcu_position', 'first_step_time', 'first_clock', 'start_position', 'last_step_time', 'last_clock', 'data', 'step_distance'}


In [2]:
def get_field(field_name: str, data):
    def _f(row):
        if 'q' not in row or row['q'] != 'status':
            return False
        return field_name in row['params']['status']

    def _m(row):
        return row['params']['eventtime'], row['params']['status'][field_name]
    
    
    return list(map(_m, filter(_f, data)))
analyze_field = get_field

In [128]:
analyze_field('heater_bed', data)

[(138.604529322, {'temperature': 35.95}),
 (139.105956197, {'temperature': 35.94}),
 (139.858566822, {'temperature': 35.93}),
 (140.109373801, {'temperature': 35.92}),
 (140.861922186, {'temperature': 35.91}),
 (141.363362394, {'temperature': 35.9}),
 (142.115582081, {'temperature': 35.88}),
 (142.3668238, {'temperature': 35.89}),
 (143.122195466, {'temperature': 35.88}),
 (143.373210362, {'temperature': 35.87}),
 (143.623853383, {'temperature': 35.86}),
 (144.375326768, {'temperature': 35.85}),
 (144.877123226, {'temperature': 35.84}),
 (145.128323643, {'temperature': 35.83}),
 (146.13086583, {'temperature': 35.84}),
 (146.632630048, {'temperature': 35.83}),
 (147.385596559, {'temperature': 35.81}),
 (147.636227809, {'temperature': 35.8}),
 (148.388658954, {'temperature': 35.79}),
 (148.889807912, {'temperature': 35.78}),
 (149.642425099, {'temperature': 35.77}),
 (150.14358661, {'temperature': 35.76}),
 (151.146041505, {'temperature': 35.75}),
 (152.148796348, {'temperature': 35.74})

In [170]:
analyze_field('tmc2209 stepper_x', data)

[(185.747351804, {'phase_offset_position': 404.6875}),
 (185.997938263,
  {'phase_offset_position': 19.6875,
   'drv_status': {'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (186.750428679, {'phase_offset_position': -115.48750000000001}),
 (187.003870346, {'phase_offset_position': 269.51250000000005}),
 (188.006857064, {'phase_offset_position': 274.51250000000005}),
 (262.214450109,
  {'drv_status': {'olb': 1, 'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (263.217340681, {'drv_status': {'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (279.265253592,
  {'drv_status': {'olb': 1, 'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (280.26839005, {'drv_status': {'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (328.416640656,
  {'drv_status': {'olb': 1, 'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (329.420093937, {'drv_status': {'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (353.505421376,
  {'drv_status': {'olb': 1, 'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (354.507470907, {'drv_status': {'cs_ac

In [171]:
analyze_field('tmc2209 extruder', data)

[(465.080553885, {'drv_status': {'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (469.093554821, {'drv_status': {'cs_actual': 25, 'stealth': 1}}),
 (483.143897368, {'drv_status': {'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (484.147603097,
  {'drv_status': {'ola': 1, 'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (485.150380232, {'drv_status': {'cs_actual': 25, 'stealth': 1}}),
 (520.254591625, {'drv_status': {'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (521.258142822, {'drv_status': {'cs_actual': 25, 'stealth': 1}}),
 (556.376289007, {'drv_status': {'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (557.379293329, {'drv_status': {'cs_actual': 25, 'stealth': 1}}),
 (564.399550306, {'drv_status': {'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (565.403267232, {'drv_status': {'cs_actual': 25, 'stealth': 1}}),
 (620.554105128, {'drv_status': {'cs_actual': 25, 'stealth': 1, 'stst': 1}}),
 (621.306464451, {'drv_status': {'cs_actual': 25, 'stealth': 1}}),
 (652.388312199, {'drv_status': {'cs_act

In [130]:
analyze_field('mcu', data)

[(138.854767708,
  {'last_stats': {'mcu_awake': 0.0,
    'mcu_task_avg': 1e-06,
    'mcu_task_stddev': 1e-06,
    'bytes_write': 8140,
    'bytes_read': 19510,
    'bytes_retransmit': 9,
    'bytes_invalid': 0,
    'send_seq': 422,
    'receive_seq': 422,
    'retransmit_seq': 2,
    'srtt': 0.001,
    'rttvar': 0.0,
    'rto': 0.025,
    'ready_bytes': 0,
    'upcoming_bytes': 0,
    'freq': 400026075}}),
 (139.858566822,
  {'last_stats': {'mcu_awake': 0.0,
    'mcu_task_avg': 1e-06,
    'mcu_task_stddev': 1e-06,
    'bytes_write': 8146,
    'bytes_read': 19631,
    'bytes_retransmit': 9,
    'bytes_invalid': 0,
    'send_seq': 423,
    'receive_seq': 423,
    'retransmit_seq': 2,
    'srtt': 0.001,
    'rttvar': 0.0,
    'rto': 0.025,
    'ready_bytes': 0,
    'upcoming_bytes': 0,
    'freq': 400026075}}),
 (140.861922186,
  {'last_stats': {'mcu_awake': 0.0,
    'mcu_task_avg': 1e-06,
    'mcu_task_stddev': 1e-06,
    'bytes_write': 8152,
    'bytes_read': 19737,
    'bytes_retransmi

In [131]:
analyze_field('extruder', data)

[(138.35359526, {'temperature': 25.19}),
 (138.604529322, {'temperature': 25.18}),
 (139.607664218, {'temperature': 25.19}),
 (140.109373801, {'temperature': 25.18}),
 (140.360024999, {'temperature': 25.17}),
 (140.861922186, {'temperature': 25.18}),
 (141.613939686, {'temperature': 25.19}),
 (142.871484633, {'temperature': 25.18}),
 (143.122195466, {'temperature': 25.17}),
 (144.124935831, {'temperature': 25.18}),
 (144.626681872, {'temperature': 25.17}),
 (144.877123226, {'temperature': 25.16}),
 (146.13086583, {'temperature': 25.17}),
 (146.381357236, {'temperature': 25.15}),
 (147.385596559, {'temperature': 25.14}),
 (147.887338798, {'temperature': 25.15}),
 (148.639511871, {'temperature': 25.14}),
 (148.889807912, {'temperature': 25.15}),
 (149.391576089, {'temperature': 25.14}),
 (150.39423536, {'temperature': 25.13}),
 (152.148796348, {'temperature': 25.12}),
 (152.649797807, {'temperature': 25.11}),
 (153.151204994, {'temperature': 25.12}),
 (153.401599369, {'temperature': 25.1

In [132]:
analyze_field('toolhead', data)

[(138.35359526, {'estimated_print_time': 2866.9793812575}),
 (138.604529322, {'estimated_print_time': 2867.23033114}),
 (138.854767708, {'estimated_print_time': 2867.4805858375}),
 (139.105956197, {'estimated_print_time': 2867.7317907025}),
 (139.35658651, {'estimated_print_time': 2867.982437355}),
 (139.607664218, {'estimated_print_time': 2868.23353143}),
 (139.858566822, {'estimated_print_time': 2868.48445039}),
 (140.109373801, {'estimated_print_time': 2868.73527372}),
 (140.360024999, {'estimated_print_time': 2868.9859412575}),
 (140.610683801, {'estimated_print_time': 2869.23661714}),
 (140.861922186, {'estimated_print_time': 2869.4878719075}),
 (141.113155571, {'estimated_print_time': 2869.739121675}),
 (141.363362394, {'estimated_print_time': 2869.98934481}),
 (141.613939686, {'estimated_print_time': 2870.2399386925}),
 (141.865075884, {'estimated_print_time': 2870.4910912675}),
 (142.115582081, {'estimated_print_time': 2870.7416138}),
 (142.3668238, {'estimated_print_time': 287

In [173]:
analyze_field('motion_report', data)

[(185.997938263,
  {'live_position': [386.9816825250067, 0.0, 0.0, 0.0],
   'live_velocity': 30.0}),
 (186.248777481, {'live_position': [1.6495346250072545, 0.0, 0.0, 0.0]}),
 (186.499738523,
  {'live_position': [9.94375, 0.0, 0.0, 0.0],
   'live_velocity': 15.000000000000002}),
 (186.750428679,
  {'live_position': [4.37872535000688, 0.0, 0.0, 0.0], 'live_velocity': 15.0}),
 (187.003870346,
  {'live_position': [0.0, -129.53690305001638, 0.0, 0.0],
   'live_velocity': 30.0}),
 (187.254792533,
  {'live_position': [0.0, 259.97794225371695, 0.0, 0.0],
   'live_velocity': 9.39313500020944}),
 (187.50589446,
  {'live_position': [0.0, 250.05625, 0.0, 0.0],
   'live_velocity': 15.000000000000002}),
 (188.006857064,
  {'live_position': [0.0, 258.51086737499134, 0.0, 0.0],
   'live_velocity': 15.0}),
 (188.258129616,
  {'live_position': [0.0, 260.0, 598.4305323500002, 0.0],
   'live_velocity': 10.0}),
 (188.50854696, {'live_position': [0.0, 260.0, 1.4565782750041762, 0.0]}),
 (188.758909355, {'l

In [176]:
analyze_field('gcode_move', data)

[(185.747351804,
  {'position': [390.0, 0.0, 0.0, 0.0],
   'gcode_position': [390.0, 0.0, 0.0, 0.0]}),
 (185.997938263,
  {'position': [10.0, 0.0, 0.0, 0.0],
   'gcode_position': [10.0, 0.0, 0.0, 0.0]}),
 (186.750428679,
  {'position': [0.0, -130.0, 0.0, 0.0],
   'gcode_position': [0.0, -130.0, 0.0, 0.0]}),
 (187.003870346,
  {'position': [0.0, 250.0, 0.0, 0.0],
   'gcode_position': [0.0, 250.0, 0.0, 0.0]}),
 (188.006857064,
  {'position': [0.0, 260.0, 600.0, 0.0],
   'gcode_position': [0.0, 260.0, 600.0, 0.0]}),
 (188.258129616,
  {'position': [0.0, 260.0, 10.0, 0.0],
   'gcode_position': [0.0, 260.0, 10.0, 0.0]}),
 (189.511375918,
  {'position': [0.0, 260.0, 0.0, 0.0],
   'gcode_position': [0.0, 260.0, 0.0, 0.0]}),
 (461.320185189,
  {'position': [390.0, 260.0, 0.0, 0.0],
   'gcode_position': [390.0, 260.0, 0.0, 0.0]}),
 (461.571362116,
  {'position': [10.0, 260.0, 0.0, 0.0],
   'gcode_position': [10.0, 260.0, 0.0, 0.0]}),
 (462.323374824,
  {'position': [0.0, -130.0, 0.0, 0.0],
   '

In [178]:
analyze_field('display_status', data)

[(199.789637997, {'progress': 0.0012760656054961537}),
 (346.980192941, {'progress': 0.0014393536213653296}),
 (461.320185189, {'progress': 0.0016994049058977213}),
 (465.080553885, {'progress': 0.0030903768929314433}),
 (473.616274611, {'progress': 0.003858435337945716}),
 (480.88725565, {'progress': 0.004154772848226813}),
 (481.138263254, {'progress': 0.004432967245633557}),
 (481.640038046, {'progress': 0.005485267792346026}),
 (483.394749243, {'progress': 0.006422661957520925}),
 (483.645502628, {'progress': 0.007529391842856452}),
 (484.397973513, {'progress': 0.008636121728191979}),
 (484.648886899, {'progress': 0.009748899317818955}),
 (485.150380232, {'progress': 0.011436208815133775}),
 (485.901857159, {'progress': 0.0125308432918864}),
 (486.152892211, {'progress': 0.013637573177221926}),
 (486.403209242, {'progress': 0.014193961972035415}),
 (486.653974398, {'progress': 0.01528254874449659}),
 (487.406669658, {'progress': 0.016401374038415018}),
 (487.657623408, {'progress'

# Load data

In [ ]:
from dataclasses import dataclass
from typing import Optional

@dataclass
class Data:
    time: Optional[float]
    

# Next steps

* Starttimestamp um mit anderen Sensoren zusammenzulegen?
* --Gemeinsamer Temperatursensor an MCU und Arduino!?--
* Arduino + Sensorik 5 Std
    - Gassensor (Halterung?, Wo?)
    - Temperatursensoren
    - Magnetometer & Co -> An Bett?
* Kamera + Mikro 1 std